In [1]:
import os
import tensorflow as tf

# Set CUDA device order and visible devices
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7,8,9"

# Set the device
device = '/cpu:0'
if tf.config.experimental.list_physical_devices('GPU'):
    try:
        # Restrict TensorFlow to only use the second GPU
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            tf.config.experimental.set_visible_devices(gpus[9], 'GPU')
            device = '/gpu:9'
    except RuntimeError as e:
        print(e)

print("device", device)


2024-08-27 21:43:49.740917: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-27 21:43:49.755755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-27 21:43:49.769211: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-27 21:43:49.773271: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-27 21:43:49.785753: I tensorflow/core/platform/cpu_feature_guar

device /gpu:9


Without Random Noisy images

In [1]:
# import numpy as np
# import random
# from PIL import Image
# from tqdm import tqdm
# import matplotlib.pyplot as plt
# import h5py

# class ImageProcessor:
#     """Image generation class"""    
    
#     def __init__(self, 
#                 num_to_generate,
#                 tiff_path, 
#                 dark_noise_path, 
#                 canvas_size = (64, 64), 
#                 max_electron_hits = 3):
#         """Initializes the ImageProcessor class

#         Args:
#             num_to_generate (int): Number of images to generate
#             tiff_path (str): Location of the tiff file that contains the electron hits
#             dark_noise_path (str): Path to the noisy data
#             canvas_size (tuple, optional): Size of the canvas to place the images on. Defaults to (64, 64).
#             max_electron_hits (int, optional): Maximum number of electron hits to place on the canvas. Defaults to 272.
#         """        
        
#         self.tiff_path = tiff_path
#         self.dark_noise_path = dark_noise_path
#         self.canvas_size = canvas_size
#         self.num_to_generate = num_to_generate
#         self.max_electron_hits = max_electron_hits
        
#         # Loads the images from the tiff file
#         self.images = self.load_images_from_tiff(tiff_path)
#         self.dark_noise_images = self.load_images_from_tiff(dark_noise_path)
        
#         # Creates a dark stack of the same size as the canvas
#         self.dark_noise = self.dark_stack(self.canvas_size[0])
    
#     def load_images_from_tiff(self, tiff_path):
#         """Loads the images from a tiff file

#         Args:
#             tiff_path (str): Path to the tiff file

#         Returns:
#             list: List of images
#         """        
#         with Image.open(tiff_path) as img:
#             images = []
#             for i in range(img.n_frames):
#                 img.seek(i)
#                 images.append(np.array(img))
#             return images
        
#     def noisy(self, noise_typ, image): 
#         """Adds noise to the images
        
#         Args:
#             noise_typ (str): Type of noise to add
#             image (numpy array): Image to add noise to
        
#         Returns:
#             numpy array: Noisy image
#         """
#         if noise_typ == "gauss":
#             row, col = image.shape
#             mean = 0
#             var = 0.0001
#             sigma = var**0.5
#             threshold = 8
#             gauss = np.random.normal(mean, sigma, (row, col))
#             tnoisy = image + gauss

#             tnoisy[tnoisy < threshold] = 0
#             noisy = np.round(tnoisy)
#             return noisy

#     def deadcorr(self, image):
#         """Corrects the dead pixel within the dark reference frame by interpolating from near positions.
        
#         Args:
#             image (numpy array): Image to correct
#         """        
#         temp = image.copy()
#         temp[:, 248] = 0.5 * temp[:, 247] + 0.5 * temp[:, 246]
#         return temp

#     def dark_stack(self, imgsize):
#         """Creates a dark stack of the same size as the canvas.
        
#         Args:
#             imgsize (int): Size of the images in the stack
#         """        
#         dark_noise_stack_corrected = [self.deadcorr(image) for image in self.dark_noise_images]
#         dark_noise_stack_cropped = [image[512:512+imgsize, 512:512+imgsize] for image in dark_noise_stack_corrected]
#         return dark_noise_stack_cropped

#     def place_image_on_canvas(self, positions=3, box_size=2):
#         """Places the electron hits on the canvas.
        
#         Args:
#             positions (int): Number of electron hits to place on the canvas
#         """        
#         canvas = np.zeros(self.canvas_size, dtype=np.uint8)
#         height, width = self.images[0].shape
#         max_x = self.canvas_size[1]
#         max_y = self.canvas_size[0]
#         bounding_boxes = []
#         centers = []
#         index_ = []
#         bounding_boxes_training = np.zeros((self.max_electron_hits, 5), dtype=np.float32)
#         centers_training = np.zeros((self.max_electron_hits, 3), dtype=np.float32)
        
#         for i in range(positions):
#             # Ensure the top-left corner is within bounds
#             x = random.randint(0, max_x - width)
#             y = random.randint(0, max_y - height)
#             index = random.randint(0, len(self.images) - 1)
#             hit = self.images[index]

#             y_min = y
#             y_max = y + height
#             x_min = x 
#             x_max = x + width
            
#             # Calculate the exact center based on pixel intensity
#             total_intensity = np.sum(hit)
#             if total_intensity > 0:
#                 y_coords, x_coords = np.indices(hit.shape)
#                 x_center = x + np.sum(x_coords * hit) / total_intensity
#                 y_center = y + np.sum(y_coords * hit) / total_intensity
#             else:
#                 x_center = x + width / 2
#                 y_center = y + height / 2
            
#             canvas[y_min:y_max, x_min:x_max] = hit
#             bounding_boxes.append((x_center-1, y_center-1, x_center+1, y_center+1))
#             bounding_boxes_training[i, 0] = 1
#             bounding_boxes_training[i, 1:] = [x_center-1, y_center-1, x_center+1, y_center+1]
#             centers.append((x_center, y_center))
#             centers_training[i, 0] = 1
#             centers_training[i, 1:] = [x_center, y_center]
#             index_.append(index)
        
#         canvas = self.noisy('gauss', canvas)
#         noise_int = np.random.randint(len(self.dark_noise))
#         canvas = canvas + self.dark_noise[noise_int]
            
#         return (canvas, bounding_boxes, bounding_boxes_training, centers, centers_training, index_, positions, noise_int)

#     def generate_multiple_images(self):
#         """Generates multiple images"""        
#         results = []
#         for i in tqdm(range(self.num_to_generate), desc="Generating images"):
#             positions = 3  # Set to exactly 3 electron hits
#             results.append(self.place_image_on_canvas(positions))
#         return results

#     def save_to_h5(self, data, filename):
#         """Saves data to an HDF5 file
        
#         Args:
#             data (list): List of data to save
#             filename (str): Path to the HDF5 file
#         """
#         with h5py.File(filename, 'w') as h5_file:
#             theimages = h5_file.create_dataset('images', shape=(self.num_to_generate, 64, 64), dtype='uint8')
#             thecenters = h5_file.create_dataset('centers_training', shape=(self.num_to_generate, 3, 3), dtype='float32')
#             for i, item in enumerate(data):
#                 theimages[i] = item[0]
#                 thecenters[i] = item[4]


# tiff_path = '/home/da886/ElectronCountingProject/200kV_98000electron.tif'
# dark_noise_path = '/home/da886/ElectronCountingProject/1000fps_fullRolling.tif'

# processor = ImageProcessor(100000, tiff_path, dark_noise_path, max_electron_hits=3)
# data = processor.generate_multiple_images()
# processor.save_to_h5(data, '/home/da886/Electron Counting Extended Arch/Data/100K_3electronsOnly.h5')


Generating images: 100%|██████████| 100000/100000 [00:16<00:00, 5885.27it/s]


With Random Noisy Images

In [1]:
import numpy as np
import random
from PIL import Image
from tqdm import tqdm
import h5py

class ImageProcessor:
    """Image generation class"""    
    
    def __init__(self, 
                num_to_generate,
                tiff_path, 
                dark_noise_path, 
                canvas_size = (64, 64), 
                max_electron_hits = 3):
        """Initializes the ImageProcessor class

        Args:
            num_to_generate (int): Number of images to generate
            tiff_path (str): Location of the tiff file that contains the electron hits
            dark_noise_path (str): Path to the noisy data
            canvas_size (tuple, optional): Size of the canvas to place the images on. Defaults to (64, 64).
            max_electron_hits (int, optional): Maximum number of electron hits to place on the canvas. Defaults to 272.
        """        
        
        self.tiff_path = tiff_path
        self.dark_noise_path = dark_noise_path
        self.canvas_size = canvas_size
        self.num_to_generate = num_to_generate
        self.max_electron_hits = max_electron_hits
        
        # Loads the images from the tiff file
        self.images = self.load_images_from_tiff(tiff_path)
        self.dark_noise_images = self.load_images_from_tiff(dark_noise_path)
        
        # Creates a dark stack of the same size as the canvas
        self.dark_noise = self.dark_stack(self.canvas_size[0])
    
    def load_images_from_tiff(self, tiff_path):
        """Loads the images from a tiff file

        Args:
            tiff_path (str): Path to the tiff file

        Returns:
            list: List of images
        """        
        with Image.open(tiff_path) as img:
            images = []
            for i in range(img.n_frames):
                img.seek(i)
                images.append(np.array(img))
            return images
        
    def noisy(self, noise_typ, image): 
        """Adds noise to the images
        
        Args:
            noise_typ (str): Type of noise to add
            image (numpy array): Image to add noise to
        
        Returns:
            numpy array: Noisy image
        """
        # if noise_typ == "gauss":
        #     row, col = image.shape
        #     mean = 0
        #     var = 0.0001
        #     sigma = var**0.5
        #     threshold = 8
        #     gauss = np.random.normal(mean, sigma, (row, col))
        #     tnoisy = image + gauss

        #     tnoisy[tnoisy < threshold] = 0
        #     noisy = np.round(tnoisy)
        #     return np.clip(noisy, 0, 255).astype(np.uint8)
        if noise_typ == "gauss":
            row,col,= image.shape
            mean = 0             #taken from sparse frames of celeritas histogram . mean = -0.17  .RMSD = 1.319
            var = 0.0001           #square of the standard deviation from sparse frames histogram
            sigma = var**0.5
            threshold = 8 #8ADU
            gauss = np.random.normal(mean,sigma,(row,col))     #thresholding instead of adding noise
            tnoisy = image + gauss                            #just making sure they're not exactly the same

            for x in range(row):                    #Noise thresholding so that only positive noise is added to the empty pixels
                    for y in range(col):
                                                        #each pixel should be an integer, not a floating point. Round each pixel after noise is added.
                        if tnoisy[x][y] < threshold:      #trying not to wipe out smaller contributions
                            tnoisy[x][y] = 0

            noisy = np.round(tnoisy) #+ gauss)    #adding noise after thresholding to recreate the image
            return noisy

    def deadcorr(self, image):
        """Corrects the dead pixel within the dark reference frame by interpolating from near positions.
        
        Args:
            image (numpy array): Image to correct
        """        
        temp = image.copy()
        temp[:, 248] = 0.5 * temp[:, 247] + 0.5 * temp[:, 246]
        return temp

    def dark_stack(self, imgsize):
        """Creates a dark stack of the same size as the canvas.
        
        Args:
            imgsize (int): Size of the images in the stack
        """        
        dark_noise_stack_corrected = [self.deadcorr(image) for image in self.dark_noise_images]
        dark_noise_stack_cropped = [image[512:512+imgsize, 512:512+imgsize] for image in dark_noise_stack_corrected]
        return dark_noise_stack_cropped

    
    # def place_image_on_canvas(self, positions=3, intensity_range=(10, 255)):
    def place_image_on_canvas(self, positions=3, intensity_range=(200, 255)):
        """Places the electron hits on the canvas with specified intensity range.
        
        Args:
            positions (int): Number of electron hits to place on the canvas.
            intensity_range (tuple): The minimum and maximum intensity of the electron hits to place on the canvas.
        """
        canvas = np.zeros(self.canvas_size, dtype=np.uint8)
        height, width = self.images[0].shape
        max_x = self.canvas_size[1]
        max_y = self.canvas_size[0]
        bounding_boxes = []
        centers = []
        index_ = []
        bounding_boxes_training = np.zeros((self.max_electron_hits, 5), dtype=np.float32)
        centers_training = np.zeros((self.max_electron_hits, 3), dtype=np.float32)
        
        for i in range(positions):
            # Select an electron hit with intensity within the specified range
            while True:
                index = random.randint(0, len(self.images) - 1)
                hit = self.images[index]
                if intensity_range[0] <= np.max(hit) <= intensity_range[1]:
                    break
            
            # Random position for placing the electron hit
            x = random.randint(0, max_x - width)
            y = random.randint(0, max_y - height)

            y_min = y
            y_max = y + height
            x_min = x 
            x_max = x + width
            
            # Calculate the exact center based on pixel intensity
            total_intensity = np.sum(hit)
            if total_intensity > 0:
                y_coords, x_coords = np.indices(hit.shape)
                x_center = x + np.sum(x_coords * hit) / total_intensity
                y_center = y + np.sum(y_coords * hit) / total_intensity
            else:
                x_center = x + width / 2
                y_center = y + height / 2
            
            canvas[y_min:y_max, x_min:x_max] = hit
            bounding_boxes.append((x_center-1, y_center-1, x_center+1, y_center+1))
            bounding_boxes_training[i, 0] = 1
            bounding_boxes_training[i, 1:] = [x_center-1, y_center-1, x_center+1, y_center+1]
            centers.append((x_center, y_center))
            centers_training[i, 0] = 1
            centers_training[i, 1:] = [x_center, y_center]
            index_.append(index)
        
        canvas = self.noisy('gauss', canvas)
        noise_int = np.random.randint(len(self.dark_noise))
        canvas = canvas + self.dark_noise[noise_int]
            
        return (canvas, bounding_boxes, bounding_boxes_training, centers, centers_training, index_, positions, noise_int)

    
    #### USE THIS BLOCK TO GENERATE FIXED SET
    # #def generate_multiple_images(self, intensity_range=(10, 255)):
    # def generate_multiple_images(self, intensity_range=(200, 255)):
    #     """Generates multiple images with specified intensity range for the objects."""        
    #     results = []
    #     for i in tqdm(range(self.num_to_generate), desc="Generating images"):
    #         results.append(self.place_image_on_canvas(positions=3, intensity_range=intensity_range))
    #     return results
    
    #### USE THIS BLOCK TO GENERATE PADDED SET
    def generate_multiple_images(self , intensity_range=(200, 255)):
            """Generates multiple images"""        
            results = []
            for i in tqdm(range(self.num_to_generate), desc="Generating images"):
                positions = random.randint(1, self.max_electron_hits)
                results.append(self.place_image_on_canvas(positions,intensity_range=intensity_range))
            return results
        
    def generate_noise_only_images(self, num_images):
        """Generates images containing only noise, with no electron hits.

        Args:
            num_images (int): Number of noise-only images to generate.

        Returns:
            list: List of generated noise-only images.
        """
        noise_images = []
        for _ in tqdm(range(num_images), desc="Generating noise-only images"):
            # Create an empty canvas
            canvas = np.zeros(self.canvas_size, dtype=np.uint8)

            # Apply Gaussian noise
            canvas = self.noisy('gauss', canvas)

            # Add dark noise
            noise_int = np.random.randint(len(self.dark_noise))
            canvas = canvas + self.dark_noise[noise_int]

            noise_images.append(canvas)

        return noise_images

    def save_mixed_images_to_h5(self, data_with_objects, noise_images, filename):
        """Mixes object-containing images with noise-only images and saves to an HDF5 file
        
        Args:
            data_with_objects (list): List of data with objects to save.
            noise_images (list): List of noise-only images to save.
            filename (str): Path to the HDF5 file.
        """
        total_images = len(data_with_objects) + len(noise_images)
        combined_data = []

        # Add labels: 1 for images with objects, 0 for noise-only images
        for item in data_with_objects:
            combined_data.append((item[0], item[4], 1))  # 1 indicates it contains objects
        
        for noise_image in noise_images:
            combined_data.append((noise_image, np.zeros((3, 3)), 0))  # 0 indicates noise-only
        
        # Shuffle the combined data
        random.shuffle(combined_data)

        # Save to HDF5 file
        with h5py.File(filename, 'w') as h5_file:
            theimages = h5_file.create_dataset('images', shape=(total_images, 64, 64), dtype='uint8')
            thecenters = h5_file.create_dataset('centers_training', shape=(total_images, 3, 3), dtype='float32')
            labels = h5_file.create_dataset('labels', shape=(total_images,), dtype='uint8')
            
            for i, (image, center, label) in enumerate(combined_data):
                theimages[i] = image
                thecenters[i] = center
                labels[i] = label

# Example usage:
tiff_path = '/home/da886/ElectronCountingProject/200kV_98000electron.tif'
dark_noise_path = '/home/da886/ElectronCountingProject/1000fps_fullRolling.tif'

# Creating an instance of ImageProcessor
processor = ImageProcessor(9000, tiff_path, dark_noise_path, max_electron_hits=3)

# Generating images with objects within a specific intensity range
data_with_intensity_range = processor.generate_multiple_images(intensity_range=(200, 255))

# Generating noise-only images
noise_only_images = processor.generate_noise_only_images(1000)

# Save mixed dataset to an HDF5 file
processor.save_mixed_images_to_h5(data_with_intensity_range, noise_only_images, '/home/da886/Electron Counting Extended Arch/Data/Padded_Mixed10K_3.h5')


Generating noise-only images: 100%|██████████| 100/100 [00:00<00:00, 895.37it/s]
